<a href="https://colab.research.google.com/github/BrunoGeraldine/B3_Prediction_2023/blob/main/notebook/EDA_Yfinance_MACD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MACD→ Analise das ações utilizando as médias moveis como orientação para compra e venda.

In [1]:
# Importando as bibliotecas
import warnings
import numpy                as np
import pandas               as pd
import yfinance             as yf
import seaborn              as sns
import plotly.graph_objects as go

In [2]:
# Help function
# Configuração do padrão de visualização dos dados e graficos
# https://seaborn.pydata.org/tutorial.html
sns.set_theme(
    context='talk',
    style='ticks',
    palette='icefire',
    font_scale=.8,
    rc={'figure.figsize': (25, 15)}
)

warnings.filterwarnings("ignore")
     

In [73]:
# Escolhendo a ação
paper = 'PETR4.SA'

ticker = yf.Ticker(paper)

# Intervalo de tempo
df = ticker.history(period='10y')
#df = df['Close']


df_ticker_daily = ticker.history(period='1d', interval='5m')

df_ticker_daily = df_ticker_daily['Close']

In [74]:
info = ticker.info
acao = info['symbol']
acao

'PETR4.SA'

In [75]:
df = pd.DataFrame(df)
df_ticker_daily = pd.DataFrame(df_ticker_daily)

In [76]:
df_ticker_daily.reset_index(inplace=True)

# Quando aplicamos o Yfinance, ele acaba trazendo as informações do momento atual
# e neste momento não queremos isso
#df.drop(df.tail(1).index,inplace=True)
df.reset_index(inplace=True)

In [77]:
df.dtypes

Date            datetime64[ns, America/Sao_Paulo]
Open                                      float64
High                                      float64
Low                                       float64
Close                                     float64
Volume                                      int64
Dividends                                 float64
Stock Splits                              float64
dtype: object

In [78]:
df.drop(columns=['Dividends', 'Stock Splits'], inplace=True)

In [79]:
# Criando um novo Df com os dados historicos
df_ticker = df

In [80]:
# Remove a ultima data do df_daily
df_remove_ticket = df_ticker.loc[(df_ticker['Date'] == pd.to_datetime('today').normalize())]
df_ticker = df_ticker.drop(df_remove_ticket.index)
df_ticker

,Date,Open,High,Low,Close,Volume
0,2013-05-13 00:00:00-03:00,7.289848,7.304568,7.131613,7.172092,21577600
1,2013-05-14 00:00:00-03:00,7.186810,7.242008,7.083773,7.194169,30643700
2,2013-05-15 00:00:00-03:00,7.175771,7.227290,7.113213,7.201530,21869300
3,2013-05-16 00:00:00-03:00,7.183132,7.389205,7.161052,7.267769,25067200
4,2013-05-17 00:00:00-03:00,7.326647,7.378165,7.282489,7.319287,19040100
...,...,...,...,...,...,...
2481,2023-05-08 00:00:00-03:00,24.450001,24.629999,24.209999,24.510000,59352300
2482,2023-05-09 00:00:00-03:00,24.410000,24.799999,24.240000,24.590000,33978200
2483,2023-05-10 00:00:00-03:00,24.590000,24.820000,24.330000,24.530001,41500700
2484,2023-05-11 00:00:00-03:00,24.129999,25.480000,23.930000,25.430000,136878100


In [81]:
df_ticker['Date'] = pd.to_datetime(df_ticker['Date'], format='%Y-%m-%d')

In [82]:
df_ticker['Date'] = pd.to_datetime(df_ticker['Date'], utc=True).dt.date
df_ticker

,Date,Open,High,Low,Close,Volume
0,2013-05-13,7.289848,7.304568,7.131613,7.172092,21577600
1,2013-05-14,7.186810,7.242008,7.083773,7.194169,30643700
2,2013-05-15,7.175771,7.227290,7.113213,7.201530,21869300
3,2013-05-16,7.183132,7.389205,7.161052,7.267769,25067200
4,2013-05-17,7.326647,7.378165,7.282489,7.319287,19040100
...,...,...,...,...,...,...
2481,2023-05-08,24.450001,24.629999,24.209999,24.510000,59352300
2482,2023-05-09,24.410000,24.799999,24.240000,24.590000,33978200
2483,2023-05-10,24.590000,24.820000,24.330000,24.530001,41500700
2484,2023-05-11,24.129999,25.480000,23.930000,25.430000,136878100


In [83]:
df_ticker

,Date,Open,High,Low,Close,Volume
0,2013-05-13,7.289848,7.304568,7.131613,7.172092,21577600
1,2013-05-14,7.186810,7.242008,7.083773,7.194169,30643700
2,2013-05-15,7.175771,7.227290,7.113213,7.201530,21869300
3,2013-05-16,7.183132,7.389205,7.161052,7.267769,25067200
4,2013-05-17,7.326647,7.378165,7.282489,7.319287,19040100
...,...,...,...,...,...,...
2481,2023-05-08,24.450001,24.629999,24.209999,24.510000,59352300
2482,2023-05-09,24.410000,24.799999,24.240000,24.590000,33978200
2483,2023-05-10,24.590000,24.820000,24.330000,24.530001,41500700
2484,2023-05-11,24.129999,25.480000,23.930000,25.430000,136878100


In [87]:
df_ticker.to_parquet('/home/bruno/repos/B3_Prediction_2023/dataset/Petro_13_to_23.TXT')
